# Project 3: Web APIs and NLP <br>

#### Brandie Hatch

### Exploratory Data Analysis, Cleaning, and Feature Engineering

## Problem



Learn with Chewie presents:
Web API and NLP Services

What do the users of Reddit consider dog CARE vs. dog TRAINING?

__Data Dictionary__

Features used listed below:

| **Feature**        | **Type** | **Dataset**  | **Description**                                                                                                                                                                                                              |
|--------------------|----------|--------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **id**             | _int64_  | train & test | ID number used for Kaggle Submission.                                                                                                                                                                                        |
| **ms_subclass**    | _object_ | train & test | Identifies the type of dwelling involved in the sale.                                                                                                                                                                        |
| **ms_zoning**      | _object_ | train & test | Identifies the general zoning classification of the sale.                                                                                                                                                                    |
| **neighborhood**   | _object_ | train & test | Physical locations within Ames city limits.                                                                                                                                                                                  |
| **n_code**         | _object_ | train & test | Neighborhood locations bundled based roughly on Census Tracts.                                                                                                                                                               |
| **percent_pop**    | _object_ | train & test | Percentage of total Black population that reported their race as Black or African American in the [2020 Census](https://mtgis-portal.geo.census.gov/arcgis/apps/MapSeries/index.html?appid=2566121a73de463995ed2b2fd7ff6eb7). |
| **condition_1**    | _object_ | train & test | Proximity to various conditions.                                                                                                                                                                                             |
| **condition_2**    | _object_ | train & test | Proximity to various conditions (if more than one is present).                                                                                                                                                               |
| **overall_qual**   | _int64_  | train & test | Rates the overall material and finish of the house.                                                                                                                                                                          |
| **overall_cond**   | _int64_  | train & test | Rates the overall condition of the house.                                                                                                                                                                                    |
| **year_built**     | _int64_  | train & test | Original construction date.                                                                                                                                                                                                  |
| **year_remod_add** | _int64_  | train & test | Remodel date (same as construction date if no remodeling or additions).                                                                                                                                                      |
| **sale_type**      | _object_ | train & test | Type of sale.                                                                                                                                                                                                                |
| **saleprice**      | _int64_  | train        | Sale price in US Dollars.                                                                                                                                                                                                    |


## Imports and Reading In Data

In [3]:
# python library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

%matplotlib inline
pd.options.display.max_columns =999

import requests
import time
import re
import nltk

In [11]:
dogtraining = pd.read_csv('./dogtraining.csv')
print(dogtraining.shape)
dogtraining.head()

(5099, 8)


,subreddit,id,author,score,title,selftext,num_comments,created_utc
0,Dogtraining,up55oy,devitosecret,1,Introducing big puppy to smaller older dog?,I have a 5.5 month old GSD who gets very excit...,0,1652487326
1,Dogtraining,up52vw,slothsandwhich,1,How do I get a cafe/brewery dog?,I am sitting at a brewery right now and all th...,0,1652487073
2,Dogtraining,up4qcr,L1Llunch,1,Not paying attention to me outside of my bedroom.,"i have a year old german shepherd, have only h...",2,1652485953
3,Dogtraining,up4b6t,Affectionate-Box-276,1,6 month old husky/ shepherd mix suddenly terri...,We adopted a husky/ shepherd mix about 3 weeks...,1,1652484573
4,Dogtraining,up49sh,Live_Damage_2902,1,why does my Dog get excited when certain peopl...,I've noticed that when my phone rings it gets ...,0,1652484449


The Dog Training data set includes 5099 observations of eight variables. 

In [10]:
dogtraining.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5099 entries, 0 to 5098
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   subreddit     5099 non-null   object
 1   id            5099 non-null   object
 2   author        5099 non-null   object
 3   score         5099 non-null   int64 
 4   title         5099 non-null   object
 5   selftext      4503 non-null   object
 6   num_comments  5099 non-null   int64 
 7   created_utc   5099 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 318.8+ KB


In [5]:
dogtraining.isnull().sum()

subreddit         0
id                0
author            0
score             0
title             0
selftext        596
num_comments      0
created_utc       0
dtype: int64

In [12]:
dogtraining.describe()

,score,num_comments,created_utc
count,5099.000000,5099.000000,5.099000e+03
mean,1.012748,0.712493,1.649657e+09
std,0.457742,1.063178,1.582885e+06
min,0.000000,0.000000,1.646968e+09
25%,1.000000,0.000000,1.648247e+09
50%,1.000000,0.000000,1.649605e+09
75%,1.000000,1.000000,1.651023e+09
max,32.000000,25.000000,1.652487e+09


In [7]:
dogcare = pd.read_csv('./dogcare.csv')
dogcare.head()

,subreddit,id,author,score,title,selftext,num_comments,created_utc
0,DogCare,uoyffw,Disastrous_Bobcat402,1,Anyone know what this could be? 7 Yr old Irish...,NaN,0,1652467478
1,DogCare,uoxknd,PerformanceStraight,1,Gum stuck in paw,[removed],0,1652465017
2,DogCare,uotky9,Routine_Heart2153,1,Nala's Crucial ACL Surgery,[removed],0,1652453786
3,DogCare,uokuoz,PrincessStorm97,1,Enrichment that doesn’t involve treats or exer...,Sad to say my oldest has heart worms. We start...,0,1652420750
4,DogCare,uohszh,LaTesora,1,New Husky/German Shepherd owner looking for ad...,Hi y'all!\n\nI just adopted a 5 year old Germa...,0,1652409660


In [8]:
dogcare.isnull().sum()

subreddit          0
id                 0
author             0
score              0
title              0
selftext        2180
num_comments       0
created_utc        0
dtype: int64

## Explore Data

## Model

## Model Evaluation

## Answer Problem, Conclusions, and Recommendations

## Sources